In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import math
import cv2
from sklearn import model_selection

In [2]:
data_path = './data/'
df = pd.read_csv(data_path + 'driving_log.csv')
steerings = df.ix[:,3]
steering_abs = np.absolute(steerings)
steering_min = steering_abs.min()
steering_max = steering_abs.max()
n_bin = 100
max_per_bin = 100
min_per_bin = 50


In [3]:
def plot_imgs(imgs, steerings):
    plt.subplots_adjust()
    count = imgs.shape[0]
    n_col = 4
    n_row = math.ceil(count / n_col)
    n_row = 2 if n_row == 1 else n_row
    fig, ax = plt.subplots(n_row, n_col, figsize=[16, 3 * n_row/2] )
    for i in range(n_row):
        for j in range(n_col):
            ij = j + i * n_col
            if ij < count:
                ax[i,j].axis('off')
                ax[i,j].set_title(steerings[ij])
                ax[i,j].imshow(imgs[ij])

In [4]:
def read_imgs(df, h, w):
    imgs = np.zeros([0, h, w, 3], dtype='uint8')
    angles = np.zeros(0, dtype='float32')
    for i,row in df.iterrows():
        _, _, filename = row[0].partition('IMG')
        img = cv2.imread(data_path + 'IMG' + filename)
        img = img[60:-20 ,:]
        img = img[..., ::-1]
        img = cv2.resize(img, (w, h))
        imgs = np.concatenate([imgs, [img]])
        angle = row[3]
        angles = np.concatenate([angles, [angle]])
    return imgs, angles

In [5]:
def random_expand_imgs(imgs, angles, to_generate):
    to_replace = False if to_generate < imgs.shape[0] else True
    indices = np.random.choice(np.arange(imgs.shape[0]), to_generate, replace=to_replace)
    ret_imgs = np.copy(imgs)
    ret_angles = np.copy(angles)
    for i in indices:
        img = imgs[i]
        angle = angles[i]
        opt = np.random.choice(3)
        if opt == 0: # blur
            img = cv2.blur(img, (3,3))
        else:
            modifier = np.random.rand() + 0.5
            modifier = 0.5
            img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
            if opt == 1:
                img[...,2] = modifier * img[...,2]
            else:
                img_indices = np.indices(img.shape)
                ys = img_indices[0]
                xs = img_indices[1]
                m = np.random.choice(np.arange(-2,2,0.1))
                b = np.random.choice(img.shape[0]) * (np.random.choice([-1, 1]))
                mask = np.copy(img)
                mask[...,2] = mask[...,2] * modifier
                sig = np.random.choice([-1, 1])
                img = np.where( sig * (xs*m  + b) > sig * ys, img, mask)

            img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)
            
        ret_imgs = np.append(ret_imgs, [img], axis=0)
        ret_angles = np.append(ret_angles, [angle], axis=0)
    return ret_imgs, ret_angles

In [6]:
def generate_x_y(h=20, w=80):
    bin_edges = np.linspace(0., 1., num=n_bin + 1)
    xs = np.zeros([0, h, w, 3], dtype='uint8')
    ys = np.zeros(0, dtype='float32')
    bin_start = 0
    for bin_end in bin_edges[1:]:
        bin_entries = df[(bin_start <= steering_abs) & (steering_abs < bin_end)]
        if bin_entries.shape[0] == 0:
            continue

        n_img = bin_entries.shape[0]
        
        if n_img > max_per_bin:
            bin_entries = bin_entries.sample(max_per_bin)
            imgs, angles = read_imgs(bin_entries, h, w)
            n_img = max_per_bin
            
        elif n_img < min_per_bin:
            imgs, angles = read_imgs(bin_entries, h, w)
            
            to_generate = min_per_bin - n_img
            imgs, angles = random_expand_imgs(imgs, angles, to_generate)
            n_img = min_per_bin
            
        else:
            imgs, angles = read_imgs(bin_entries, h, w)
            
        xs = np.concatenate([xs, imgs])
        ys = np.concatenate([ys, angles])
        bin_start = bin_end
    return xs, ys

In [7]:
xs, ys = generate_x_y()

In [8]:
xs.shape, ys.shape

((5164, 20, 80, 3), (5164,))

In [9]:
np.save('./xs.npy', xs)
np.save('./ys.npy', ys)